In [10]:
import pandas as pd 

import spacy
import neuralcoref

from pathlib import Path

from email import policy
from email.parser import BytesParser
import json
import nltk.data
import re
import os
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)
print("here")
#finding the files that has the keyword like meeting or conference in their text. 
hitlist=[]

user_input = input('What is the name of your directory')
print("here")
directory = os.listdir(user_input)
print("here")

###YOU CAN ADD MORE TRAVEL RELATED KEYWORDS HERE!!!!
keywords=['travel','landed','trip','flying','visit','journeyed','en route','meeting','tour','pilgrimage','holiday',
          'outing','excursion','cruise','voyage','safari','trek','jaunt','wander','event','rendez vous', 'RDV']


for fname in directory:
        if os.path.isfile(user_input + os.sep + fname):
        
            with open(user_input + os.sep + fname, 'rb') as f:
                name = f.name  # Get file name
                msg = BytesParser(policy=policy.default).parse(f)
            print(name)
     
        
            if(msg.get_body(preferencelist=('plain'))):
                text = msg.get_body(preferencelist=('plain')).get_content()    
             
            else:
                print('invalid')
                continue
           
            text = textcleaning(text)
            text = coref_resolution(text)
            #print(text)
            #split to sentence and find if the keyword is in the sentence; perform NER and add this file to hitlist only if 
            #there are a person entity and a location in that sentence
            tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

            sent= text.split('.')
            #print(sent)
            for t in sent:
                    #print('\n')
                    per=0
                   
                    locn=0
                    doc=nlp(t)#SPACY NER TOOL IS USED TO IDENTIFY IF THERE ARE PERSONS/LOCATION IN A SENTENCE
                    
                    for ent in doc.ents:
                        #print(ent.text,ent.label_)
                        
                        if ent.label_=="PERSON" or ent.label_=="NORP":
                            per=per+1
                           
                        elif ent.label_=="LOC" or ent.label_== "GPE":
                            locn=locn+1
                            
            
                    for searchstring in keywords:
                        res = bytes(searchstring, 'utf-8')
                        if ( per>0) and (locn > 0) and (res.decode("utf-8") in text):
              
                            #print('found string in file',res,fname)
                            if fname not in hitlist:
                                print('found string in file',res,fname)
                                hitlist.append(fname)
                        #break
           #prints the filenames which has some travel relevant keywords. Start reading and label trips of these files
with open("OUTPUT.txt", "a") as o:
    for item in hitlist:
        # write each item on a new line
        o.write("%s\n" % item)
   
    

here
here
here
emails\30000.eml
emails\30001.eml
emails\30002.eml
emails\30003.eml
emails\30004.eml
emails\30005.eml
found string in file b'visit' 30005.eml
emails\30006.eml
emails\30007.eml
found string in file b'travel' 30007.eml
emails\30008.eml
found string in file b'trip' 30008.eml
emails\30009.eml
emails\30010.eml
invalid
emails\30011.eml
emails\30012.eml
emails\30013.eml
emails\30014.eml
emails\30015.eml
found string in file b'trip' 30015.eml
emails\30016.eml
emails\30017.eml
emails\30018.eml
emails\30019.eml
emails\30020.eml
emails\30021.eml
found string in file b'meeting' 30021.eml
emails\30022.eml
emails\30023.eml
found string in file b'visit' 30023.eml
emails\30024.eml
emails\30025.eml
emails\30026.eml
emails\30027.eml
emails\30028.eml
found string in file b'travel' 30028.eml
emails\30029.eml
emails\30030.eml
emails\30031.eml
emails\30032.eml
found string in file b'event' 30032.eml
emails\30033.eml
emails\30034.eml
emails\30035.eml
emails\30036.eml
emails\30037.eml
emails\30

In [ ]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref is not None:
                if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                    if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                        tok_list[coref.start] = cluster.main.text + \
                            doc[coref.end-1].whitespace_
                        for i in range(coref.start+1, coref.end):
                            tok_list[i] = ""

    return "".join(tok_list)

In [9]:
def textcleaning(text):
    ###tokenising the email text
# define punctuation
    punctuations = '''!()-[]{};:'"\<>/?@#$%^&*_~'''


# Replace all occurrences of character s with an empty string

    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE)
    text = text.replace("httpgetwrap"," ")
    text= text.replace(" Sec. ", " ")
    text= text.replace(" Rep. ", " ")
    text= text.replace(" Sen. ", " ")
    text= text.replace(" U.S ", "US")
    text = re.sub('\s+',' ',text)
#text = text.replace("today",msg['Date'])
#print(text)
# remove punctuation from the string
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [8]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref is not None:
                if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                    if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                        tok_list[coref.start] = cluster.main.text + \
                            doc[coref.end-1].whitespace_
                        for i in range(coref.start+1, coref.end):
                            tok_list[i] = ""

    return "".join(tok_list)